## UAV fleet Simulation 

This is for testing propouses

In [1]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.visualization.gazebo_world import update_world
from helpers.change_coordinates import local2global,heading_to_yaw,find_spawns
from helpers.visualization.plots import plot_3d_interactive

from plan import Plan

Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [4]:
simulator ='gazebo' #'QGroundControl' # 

## Create Plans

In [5]:
offset = (0, 0, 0, 0) #east, north, up, heading
    
path = Plan.create_square_path(side_len=7,alt= 5)
home=np.array(offset[:3])

global_path=local2global(np.array(path), home,pairwise=True) 

plan = Plan.basic(wps=path,alt=5,wp_margin=0.5)


markers = {'waypoints': {'pos':global_path,'color':'green'},
           'home':      {'pos':[home],'color':'orange'}}


if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawn=find_spawns(home_position, offset)

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



Plan 'basic plan' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'Take Off' created — no connection yet 🧩
Step 'take off' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to -> (0, 7, 5)' created — no connection yet 🧩
Step 'go to -> (7, 7, 5)' created — no connection yet 🧩
Step 'go to -> (7, 0, 5)' created — no connection yet 🧩
Step 'go to -> (0, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩


Launch Simulator

In [6]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    east, north, up, heading = offset
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading))]
    world_path = os.path.expanduser(gazebo_word_path)
    del markers['home']
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Launch Vehicles

In [7]:
vehicle_cmd = f"python3 {ardupilot_vehicle_path} -v ArduCopter -I0 --sysid 1 --no-rebuild"
if simulator == 'QGroundControl':
    vehicle_cmd += f" --custom-location={spawn}"
elif simulator == 'gazebo':
    vehicle_cmd += " -f gazebo-iris"
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--', 'bash', '-c', 'pytho...>

Create UAV

In [8]:
# uav = VehicleLogic(sys_id=1,
#                     home=home,
#                     plan= plan)

uav = VehicleLogic(sys_id=1,
                    home=home,
                    plan=plan)

vehicle 1 created


In [9]:
print(uav.plan)

🕓 <Plan 'basic plan'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'Take Off'>
    🕓 <Step 'take off'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to -> (0, 0, 5)'>
    🕓 <Step 'go to -> (0, 7, 5)'>
    🕓 <Step 'go to -> (7, 7, 5)'>
    🕓 <Step 'go to -> (7, 0, 5)'>
    🕓 <Step 'go to -> (0, 0, 5)'>
  🕓 <Action 'Land'>
    🕓 <Step 'land'>


## Execute Plan

In [10]:
mission_incomplete = True
while mission_incomplete:
    mission_incomplete = not uav.act_plan()

Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: ▶️ Starting Action: Pre-Arm Check
Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: POS_HORIZ_ABS
Vehicle 1: EKF is not ready — missing: 

In [11]:
print(uav.plan)

✅ <Plan 'basic plan'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'Take Off'>
    ✅ <Step 'take off'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to -> (0, 0, 5)'>
    ✅ <Step 'go to -> (0, 7, 5)'>
    ✅ <Step 'go to -> (7, 7, 5)'>
    ✅ <Step 'go to -> (7, 0, 5)'>
    ✅ <Step 'go to -> (0, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>


## Kill all related process

In [12]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")